In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.Strategy.MVO import *
from strategy_v2.Strategy.MVO.AlphaModel import *
from strategy_v2.Strategy.MVO.RiskModel import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *
from utils.ta import *

In [2]:
instruments = [
    'META',
    'TSLA',
    'NVDA',
    'AAPL',    
    'DXJ',
    'BRK-B',
    'SPY',
    'QQQ',
    'NANC',
    #'BTC',    
    'CASH'
]

end_date = get_today(1)
start_date = pd.to_datetime(datetime(2024, 1, 3))
start_date = pd.to_datetime(datetime(2023, 2, 7))
max_leverage = 1
#vol_target = 0.25
vol_target = None
end_date

Timestamp('2025-02-03 00:00:00')

# Notes

- 2024-08-28: Tested expected return prediction with RandomForest, XGB, LGBM. None of them outperforms the SMA Model. LGBM is able to achieve a similar performance as SMA model and generally train faster.</br>

    | Measure                | ^SPX      | MVO - SMA1 | MVO - RandomForest1 | MVO - XGB1 | MVO - LGBM1 |
    |------------------------|-----------|------------|---------------------|------------|-------------|
    | Cumulative Return      | 1.173503  | 2.072994   | 1.662573            | 1.471717   | 2.023378    |
    | Annualized Return      | 0.076775  | 0.301537   | 0.221185            | 0.176047   | 0.296252    |
    | Annualized Volatility  | 0.180081  | 0.226417   | 0.240133            | 0.243953   | 0.243120    |
    | Annualized Sharpe Ratio| 0.240438  | 1.183923   | 0.781685            | 0.584416   | 1.080844    |
    | Maximum Drawdown       | -0.254251 | -0.271732  | -0.323658           | -0.380737  | -0.296535   |

    Models are shared the same hyperparameters: lookback (train days) = 10 days and gamma=10, hhi=0.2


## Leverage 
##### 1. capital (Stock MV / last leverage), 2. new leverage
2024-09-06: $169,336, 96.69%

2024-09-20: $172,168, 93.52%

2024-10-04: $176,235, 100%

2024-10-18: $177,831, 100%

2024-11-01: $175,558, 100%

2024-11-09: $181,511

2024-11-22: $187,224

2024-11-30: $188,332

2024-12-06: $194,828

2024-12-13: $211,354, CASH: $3778

2024-12-20: $198,713, CASH: $38,024 (+$20000 Capital)

2025-01-04: $192,246, CASH: $23,606

2024-01-10: $205,516, CASH: $18,562

2025-01-17: $213,405, CASH: $18,835

2025-01-24: $216,851, CASH: $11,204

2025-01-31: $228,709, CASH: $4,254


# Log

2024-11-22: If we don't fix sum of weights to constant (e.g. 1), the model would statisfy the HHI constraints by reducing the overeall weights only, but not diversing the portfolios. Still, fixing to 1 is not good because sometime we should de-leverage given signals are weak. So we can consider to add a cash element in the optimization

2024-11-30: Create benchmark portfolios (e.g. equally weighted of all stocks) (DONE)

2024-12-06: Alpha models using returns relative to something (e.g. SPX)

In [3]:
portfolio = PortfolioStandard(
    capital=228709+11204,
    name='MVOPortfolio',        
    rebalance_iter=RebalancerIter('0 0 * * Fri', 1),        
    tc_model=TransactionCostFutu(), 
    systems_style=SystemStyle.VERTICAL,
    systems=[        
        # Long terms signals => 60days return / diversified        
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(60), risk_model=RollingMeanCovNeg(60), gamma=10, hhi=0.2, confidence=1, leverage=1)], max_leverage=max_leverage, offset=60),

        # Mid terms signals => 30days return / diversified        
        # TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(30), risk_model=RollingMeanCovNeg(30), gamma=10, hhi=0.2, confidence=1, leverage=1)], max_leverage=max_leverage, offset=60),

        # Short terms signals => 5 and 10days returns / less diversified
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(10), risk_model=RollingMeanCovNeg(10), gamma=30, hhi=0.4, confidence=0.5, leverage=1)], max_leverage=max_leverage, offset=60),         
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(5), risk_model=RollingMeanCovNeg(5), gamma=30, hhi=0.4, confidence=0.5, leverage=1)], max_leverage=max_leverage, offset=60),         

        # Mean Revert signals => 2days RSI / concentrated
        # TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RSI(2,10), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=0.2)], max_leverage=max_leverage, offset=200),
        # TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RSI(2,95), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=-1)], max_leverage=max_leverage, offset=200),

        # Mean Revert signals => Double7 / concentrated
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=Double7(5), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=0.2)], max_leverage=max_leverage, offset=200),
    ]
)

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize()
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

[*********************100%***********************]  9 of 9 completed
2025-02-02 23:47:42,988 - TradingSubSystemSingle [MVO - RollingMean(60)|1] - INFO - Generating position for strategy MVO - RollingMean(60)|1 between 2023-02-06 and 2025-02-03......
100%|██████████| 521/521 [00:04<00:00, 124.25it/s]
2025-02-02 23:47:47,215 - TradingSubSystemSingle [MVO - RollingMean(60)|1] - INFO - Volatility Target = nan% | Price Volatility = 29.1% | Last Scale Factor = 1.00
2025-02-02 23:47:47,218 - TradingSubSystemSingle [MVO - RollingMean(10)|0.5] - INFO - Generating position for strategy MVO - RollingMean(10)|0.5 between 2023-02-06 and 2025-02-03......
100%|██████████| 521/521 [00:03<00:00, 134.64it/s]
2025-02-02 23:47:51,108 - TradingSubSystemSingle [MVO - RollingMean(10)|0.5] - INFO - Volatility Target = nan% | Price Volatility = 11.1% | Last Scale Factor = 1.00
2025-02-02 23:47:51,114 - TradingSubSystemSingle [MVO - RollingMean(5)|0.5] - INFO - Generating position for strategy MVO - RollingMean

,Rebalanced Portfolio,Optimized Portfolio,^SPX,Equal Weighted,MVO - RollingMean(60)|1,MVO - RollingMean(10)|0.5,MVO - RollingMean(5)|0.5,MVO - Double(5)|0.2
Measure,,,,,,,,
Annualized Return,0.555715,0.480177,0.196313,0.399216,0.541687,0.258165,0.161937,0.147892
Annualized Sharpe Ratio,2.256176,1.881103,1.329127,2.021167,2.062951,1.745985,1.034585,1.481008
Annualized Volatility,0.233753,0.240205,0.126389,0.183502,0.248847,0.131638,0.129144,0.080732
Cumulative Return,2.839510,2.438251,1.450656,2.127894,2.741564,1.636997,1.354511,1.330816
Maximum Drawdown,-0.119414,-0.145742,-0.102766,-0.121574,-0.178395,-0.111620,-0.081943,-0.040354


In [12]:
portfolio.portfolio_breakdown()
portfolio.instrument_breakdown()

In [17]:
portfolio.get_position_for_trade().tail(20)

2025-02-01 00:43:43,338 - Standard Portfolio ({self.name}) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $239,913
2025-02-01 00:43:43,340 - Standard Portfolio ({self.name}) - INFO - Generate trade position based on target capital of $239,913


,AAPL,BRK-B,DXJ,META,NANC,NVDA,QQQ,SPY,TSLA,CASH
Date,,,,,,,,,,
2025-01-02,232.0,8.0,200.0,6.0,215.0,434.0,17.0,12.0,112.0,13432.0
2025-01-03,53.0,10.0,533.0,9.0,156.0,324.0,14.0,9.0,169.0,30501.0
2025-01-06,53.0,10.0,533.0,9.0,156.0,324.0,14.0,9.0,169.0,30501.0
2025-01-07,53.0,10.0,533.0,9.0,156.0,324.0,14.0,9.0,169.0,30501.0
2025-01-08,53.0,10.0,533.0,9.0,156.0,324.0,14.0,9.0,169.0,30501.0
2025-01-10,39.0,19.0,408.0,93.0,280.0,65.0,14.0,12.0,165.0,20234.0
2025-01-13,39.0,19.0,408.0,93.0,280.0,65.0,14.0,12.0,165.0,20234.0
2025-01-14,39.0,19.0,408.0,93.0,280.0,65.0,14.0,12.0,165.0,20234.0
2025-01-15,39.0,19.0,408.0,93.0,280.0,65.0,14.0,12.0,165.0,20234.0


# Execute the Portfolio

In [18]:
executor = ExecutorFutu(is_test=False)
executor.set_portfolio(portfolio)
orders = executor.execute(px_interval='5m')
orders

2025-02-01 00:44:04,626 - ExecutorFutu - INFO - market: US
2025-02-01 00:44:04,627 - ExecutorFutu - INFO - Cancel all orders first before executing.....


2025-02-01 00:44:04,638 | 30750 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=18214795
2025-02-01 00:44:04,872 | 30750 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=1


2025-02-01 00:44:07,877 - Standard Portfolio ({self.name}) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $239,913
2025-02-01 00:44:07,879 - Standard Portfolio ({self.name}) - INFO - Generate trade position based on target capital of $239,913
2025-02-01 00:44:07,885 - ExecutorFutu - INFO - Execute Standard Portfolio ({self.name}) position based on 2025-01-31


2025-02-01 00:44:07,887 | 30750 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=2, host=127.0.0.1, port=11111, user_id=18214795


2025-02-01 00:44:07,900 - Futu - INFO - 9 Positions: US.TSLA, US.SPY, US.QQQ, US.NVDA, US.NANC, US.META, US.DXJ, US.BRK.B, US.AAPL


...

2025-02-01 00:44:32,056 - Futu - INFO - Placed Order: {'code': 'US.SPY', 'price': 609.07, 'qty': 4.0, 'trd_side': 'SELL', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2025-02-01 00:44:32,063 | 30750 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=10
2025-02-01 00:44:35,094 | 30750 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=11, host=127.0.0.1, port=11111, user_id=18214795


2025-02-01 00:44:35,375 - Futu - INFO - Placed Order: {'code': 'US.TSLA', 'price': 416.99, 'qty': 29.0, 'trd_side': 'SELL', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2025-02-01 00:44:35,377 | 30750 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=11


,code,stock_name,trd_side,order_type,order_status,order_id,qty,price,create_time,updated_time,...,remark,time_in_force,fill_outside_rth,aux_price,trail_type,trail_value,trail_spread,currency,portfolio,date
0,US.AAPL,苹果,SELL,NORMAL,SUBMITTING,4330346411400541153,66.0,239.77,2025-01-31 11:44:08,2025-01-31 11:44:08,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-31
0,US.BRK.B,伯克希尔-B,BUY,NORMAL,SUBMITTING,2481524909220134564,27.0,470.96,2025-01-31 11:44:11,2025-01-31 11:44:11,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-31
0,US.DXJ,日本对冲股票ETF-WisdomTree,SELL,NORMAL,SUBMITTING,6301086614568194790,57.0,111.28,2025-01-31 11:44:15,2025-01-31 11:44:15,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-31
0,US.META,Meta Platforms,BUY,NORMAL,SUBMITTING,7948276919224935095,79.0,698.30,2025-01-31 11:44:18,2025-01-31 11:44:18,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-31
0,US.NANC,UNUSUAL WHALES SUBVERSIVE DEMOCRATIC TRADING ETF,SELL,NORMAL,SUBMITTING,7431661177292525037,21.0,40.52,2025-01-31 11:44:22,2025-01-31 11:44:22,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-31
0,US.NVDA,英伟达,SELL,NORMAL,SUBMITTING,1716767522627742469,85.0,125.79,2025-01-31 11:44:25,2025-01-31 11:44:25,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-31
0,US.QQQ,纳指100ETF-Invesco QQQ Trust,SELL,NORMAL,SUBMITTING,4028749095755572670,16.0,530.17,2025-01-31 11:44:28,2025-01-31 11:44:28,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-31
0,US.SPY,标普500ETF-SPDR,SELL,NORMAL,SUBMITTING,7467843295703450663,4.0,609.07,2025-01-31 11:44:32,2025-01-31 11:44:32,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-31
0,US.TSLA,特斯拉,SELL,NORMAL,SUBMITTING,4504221983063158652,29.0,416.99,2025-01-31 11:44:35,2025-01-31 11:44:35,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-31


# Calibration Portfolios